In [1]:
# add parent directory to path
import sys
sys.path.append('../')
import model_utils
import torch


In [41]:
pretraind_model = 'UNI_multimodal' # 'UNI_multimodal',"UNI"
multimodal = 1 if "multimodal" in pretraind_model else 0
my_configs = {}
my_configs['pretraind_model'] = pretraind_model
my_configs['num_hidden_units'] = [128]
my_configs['dropout'] = 0.0
num_classes=10


In [42]:
model = model_utils.init_model(my_configs = my_configs,num_classes = num_classes)
model.create_model()

In [43]:
model.cuda()
model.print_model_summary()

Morph Extractor: 
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=1024, out_features=128, bias=True)
  (2): LeakyReLU(negative_slope=0.02)
)
Gene Extractor: 
Sequential(
  (0): Linear(in_features=1000, out_features=128, bias=True)
  (1): LeakyReLU(negative_slope=0.02)
)
Classifier: 
Sequential(
  (0): Linear(in_features=128, out_features=10, bias=True)
)
Combo Layer: 
Sequential(
  (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): LeakyReLU(negative_slope=0.02)
  (3): Dropout(p=0.25, inplace=False)
)


In [44]:
# make a dummy input

batch_size = 2
inp = {}
inp['image'] = torch.randn(batch_size, model.latent_dim)
if multimodal:
    inp['gene_expression'] = torch.randn(batch_size, 1000)

# put inp on cuda
inp = {k: v.cuda() for k, v in inp.items()}

out = model(inp)

print(out.shape)

torch.Size([2, 10])


In [45]:
morph_features = model.morph_extractor(inp['image'])
print("Morph features shape: ", morph_features.shape)
if hasattr(model, 'gene_extractor') and model.gene_extractor is not None:
    gene_features = model.gene_extractor(inp['gene_expression'])
    print("Gene features shape: ", gene_features.shape)
else:
    print("No gene extractor found")

if hasattr(model, 'image_extractor') and model.image_extractor is not None:
    image_features = model.image_extractor(inp['image'])
    print("Image features shape: ", image_features.shape)


Morph features shape:  torch.Size([2, 128])
Gene features shape:  torch.Size([2, 128])
